In [7]:
import pickle as pkl
import pandas as pd

# Imputer
with open('../model/imputer.pkl','rb') as fichier_pickler:
    imputer = pkl.load(fichier_pickler)


# Classification
with open('../model/modele_classification.pkl','rb') as fichier_pickler:
    model_classif = pkl.load(fichier_pickler)



# Regression
with open('../model/modele_regression_code_1.pkl','rb') as fichier_pickler:
    model_reg_1 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_2.pkl','rb') as fichier_pickler:
    model_reg_2 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_3.pkl','rb') as fichier_pickler:
    model_reg_3 = pkl.load(fichier_pickler)

with open('../model/modele_regression_code_4.pkl','rb') as fichier_pickler:
    model_reg_4 = pkl.load(fichier_pickler)



In [8]:
#
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


sub = pd.read_csv('../submissions.csv')
sub.columns = sub.columns.str.replace(' ', '_')

sub.head()

C:\Users\nagrimault\AppData\Local\Temp\ipykernel_9992\656433933.py:6: DtypeWarning: Columns (9,15,17,19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  sub = pd.read_csv('../submissions.csv')


,Date_mutation,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Nature_culture_speciale,Surface_terrain,ID
0,03/01/2022,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,24.00,1.00,NaN,NaN,NaN,1
1,03/01/2022,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,140.00,3.00,NaN,NaN,NaN,2
2,06/01/2022,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,108.00,5.00,S,NaN,649.00,3
3,05/01/2022,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,85.00,4.00,S,NaN,310.00,4
4,06/01/2022,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,99.00,5.00,S,NaN,765.00,5


In [9]:
sub['Commune'] = sub['Commune'].str.replace('-', ' ') 
sub['Commune'] = sub['Commune'].str.replace('ST', 'SAINT')  # format 'ST' et 'SAINT'
sub['Commune'] = sub['Commune'].str.replace('\'', ' ')


In [10]:

colonnes_a_imputer = ["Surface_reelle_bati","Nombre_pieces_principales","Surface_terrain","niveau_vie_commune"]


sub_2 = sub[colonnes_a_imputer]
sub_2_imputed = imputer.transform(sub_2)

sub_1 = sub.drop(columns=colonnes_a_imputer)


# Créer un DataFrame à partir du tableau NumPy imputé
sub_2_imputed_df = pd.DataFrame(sub_2_imputed, columns=colonnes_a_imputer)


sub_final = pd.concat([sub_1, sub_2_imputed_df], axis=1)
sub_final.head()

KeyError: "['niveau_vie_commune'] not in index"

In [ ]:
sub_final['Type_local'].value_counts()

# recoder ....

Type_local
Maison                                      193425
Appartement                                  61572
Dépendance                                   46331
Local industriel. commercial ou assimilé     26713
Name: count, dtype: int64

In [ ]:
# Créer la variable "day", "month" and "year" 

# NB : Utile si on la mets en 'int' et pas en objet, car le prix de l'immobilier augmente chaque année, donc plus l'année augmente plus le prix aussi
# alors qu'en 'objet' inutile car dans ce sont les data de 2022 qui vont être testés (à revoir quand même)


sub_final['Date_mutation'] = pd.to_datetime(sub_final['Date_mutation'], format='%d/%m/%Y')

# Extract the year component and create a new 'year' column
sub_final['day'] = sub_final['Date_mutation'].dt.day
sub_final['day'] = sub_final['day'].astype(object)

sub_final['month'] = sub_final['Date_mutation'].dt.month
sub_final['month'] = sub_final['month'].astype(object)

sub_final['year'] = sub_final['Date_mutation'].dt.year
sub_final['year'] = sub_final['year'].astype(object)

sub_final = sub_final.drop("Date_mutation", axis=1)
    

sub_final.head()

,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Nature_culture,Nature_culture_speciale,ID,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,day,month,year
0,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,1,24.00,1.00,470.00,3,1,2022
1,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,2,140.00,3.00,470.00,3,1,2022
2,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,3,108.00,5.00,649.00,6,1,2022
3,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,4,85.00,4.00,310.00,5,1,2022
4,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,S,NaN,5,99.00,5.00,765.00,6,1,2022


In [ ]:
## Attribuer un code type local à chaque type et créer une variable
# Créer un dictionnaire de mapping entre 'Type_local' et 'Code_type_local'
mapping_type_local = {
    'Maison': 1,
    'Appartement': 2,
    'Dépendance': 3,
    'Local industriel. commercial ou assimilé': 4
}

# Appliquer le mapping pour créer la nouvelle colonne 'Code_type_local'
sub_final['Code_type_local'] = sub_final['Type_local'].map(mapping_type_local)

# Afficher les premières lignes du DataFrame pour vérification
print(sub_final.head())

  Nature_mutation  No_voie B/T/Q Type_de_voie Code_voie           Voie  \
0           Vente    13.00   NaN          RUE      2280  DE LA LIBERTE   
1           Vente    98.00   NaN          RTE      0055   DE LA DOMBES   
2           Vente   282.00   NaN          RTE      0130    DE POISATON   
3           Vente     7.00   NaN          RUE      0031     DU CORNIER   
4           Vente     7.00   NaN          RUE      3125       DES PINS   

   Code_postal           Commune Code_departement  Code_commune  \
0      1000.00   BOURG-EN-BRESSE                1            53   
1      1480.00         SAVIGNEUX                1           398   
2      1560.00  MANTENAY-MONTLIN                1           230   
3      1150.00     VAUX-EN-BUGEY                1           431   
4      1000.00   BOURG-EN-BRESSE                1            53   

   Prefixe_de_section Section  No_plan  No_Volume 1er_lot  \
0                 NaN      AM      102        NaN     7.0   
1                 NaN      ZE 

In [ ]:
# Remplacer les valeurs manquantes dans la colonne 'Code_type_local' par une valeur par défaut (ici, 0)
sub_final['Code_type_local'].fillna(0, inplace=True)

sub_final['Code_type_local'] = sub_final['Code_type_local'].astype(int)
sub_final.dtypes

Nature_mutation                object
No_voie                       float64
B/T/Q                          object
Type_de_voie                   object
Code_voie                      object
Voie                           object
Code_postal                   float64
Commune                        object
Code_departement               object
Code_commune                    int64
Prefixe_de_section            float64
Section                        object
No_plan                         int64
No_Volume                     float64
1er_lot                        object
Surface_Carrez_du_1er_lot      object
2eme_lot                       object
Surface_Carrez_du_2eme_lot     object
3eme_lot                       object
Surface_Carrez_du_3eme_lot     object
4eme_lot                      float64
Surface_Carrez_du_4eme_lot     object
5eme_lot                      float64
Surface_Carrez_du_5eme_lot     object
Nombre_de_lots                  int64
Type_local                     object
Identifiant_

In [ ]:
sub_final.head()

,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Nature_culture,Nature_culture_speciale,ID,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,day,month,year,Code_type_local
0,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,1,24.00,1.00,470.00,3,1,2022,2
1,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,2,140.00,3.00,470.00,3,1,2022,2
2,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,3,108.00,5.00,649.00,6,1,2022,1
3,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,4,85.00,4.00,310.00,5,1,2022,1
4,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,S,NaN,5,99.00,5.00,765.00,6,1,2022,0


In [ ]:
print((sub_final['Code_type_local'] == 0).sum())

50000


In [ ]:
# Faire classif sur Code Type Local....
#lignes avec des 0=NA dans Code_type_local
lignes_0 = sub_final[sub_final['Code_type_local'] == 0]
colonne_classif=['Surface_reelle_bati', 'Nombre_pieces_principales','Surface_terrain']
predictions_classif = model_classif.predict(lignes_0[colonne_classif])

#Remplacer les valeurs manquantes dans 'Code_type_local' par les prédictions
sub_final.loc[sub_final['Code_type_local'] == 0, 'Code_type_local'] = predictions_classif
######

In [ ]:
sub_final.head()

,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Nature_culture,Nature_culture_speciale,ID,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,day,month,year,Code_type_local
0,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG-EN-BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,1,24.00,1.00,470.00,3,1,2022,2
1,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,2,140.00,3.00,470.00,3,1,2022,2
2,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY-MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,3,108.00,5.00,649.00,6,1,2022,1
3,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX-EN-BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,4,85.00,4.00,310.00,5,1,2022,1
4,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG-EN-BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,S,NaN,5,99.00,5.00,765.00,6,1,2022,1


In [ ]:
print((sub_final['Code_type_local'] == 0).sum())

0


In [ ]:
# Ajouter le prix au m2


with open('../data/prix_moyen_m2.pkl','rb') as prix_moyen_m2:
    groupe_commune = pkl.load(prix_moyen_m2)


sub_final = pd.merge(sub_final, groupe_commune[['Prix_moyen_m2']],
              left_on='Commune', right_index=True, how='left')

median_value = sub_final['Prix_moyen_m2'].median()
# Remplacez les NaN par la médiane
import numpy as np
sub_final['Prix_moyen_m2'] = sub_final['Prix_moyen_m2'].replace([np.inf, -np.inf], np.nan)
sub_final['Prix_moyen_m2'].fillna(median_value, inplace=True)


sub_final.head()

,Nature_mutation,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Type_local,Identifiant_local,Nature_culture,Nature_culture_speciale,ID,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,day,month,year,Code_type_local,Prix_moyen_m2
0,Vente,13.00,NaN,RUE,2280,DE LA LIBERTE,1000.00,BOURG EN BRESSE,1,53,NaN,AM,102,NaN,7.0,"24,10",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,1,24.00,1.00,470.00,3,1,2022,2,2177.51
1,Vente,98.00,NaN,RTE,0055,DE LA DOMBES,1480.00,SAVIGNEUX,1,398,NaN,ZE,187,NaN,1.0,"123,23",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Appartement,NaN,NaN,NaN,2,140.00,3.00,470.00,3,1,2022,2,1588.53
2,Vente,282.00,NaN,RTE,0130,DE POISATON,1560.00,MANTENAY MONTLIN,1,230,NaN,ZM,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,3,108.00,5.00,649.00,6,1,2022,1,1544.38
3,Vente,7.00,NaN,RUE,0031,DU CORNIER,1150.00,VAUX EN BUGEY,1,431,NaN,A,1613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Maison,NaN,S,NaN,4,85.00,4.00,310.00,5,1,2022,1,1924.28
4,Vente,7.00,NaN,RUE,3125,DES PINS,1000.00,BOURG EN BRESSE,1,53,NaN,CS,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,S,NaN,5,99.00,5.00,765.00,6,1,2022,1,2177.51


In [ ]:
sub_final.describe()


,No_voie,Code_postal,Code_commune,Prefixe_de_section,No_plan,No_Volume,4eme_lot,5eme_lot,Nombre_de_lots,Identifiant_local,ID,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,Code_type_local,Prix_moyen_m2
count,375777.00,378024.00,378041.00,13737.00,378041.00,0.00,2614.00,1512.00,378041.00,0.00,378041.00,378041.00,378041.00,378041.00,378041.00,378041.00
mean,616.54,53280.17,208.66,529.52,368.70,NaN,117.43,125.55,0.48,NaN,189021.00,100.36,2.84,552.41,1.72,3184.85
std,1944.43,27482.21,171.44,320.86,529.17,NaN,868.30,795.40,1.02,NaN,109131.18,483.92,2.04,6511.72,0.99,2226.48
min,1.00,1000.00,1.00,1.00,1.00,NaN,2.00,2.00,0.00,NaN,1.00,0.00,0.00,1.00,1.00,18.71
25%,7.00,31560.00,75.00,209.00,85.00,NaN,7.00,8.00,0.00,NaN,94511.00,36.00,1.00,317.00,1.00,1643.04
50%,20.00,56620.00,165.00,602.00,204.00,NaN,24.00,26.50,0.00,NaN,189021.00,75.00,3.00,470.00,1.00,2483.31
75%,66.00,76660.00,301.00,827.00,437.00,NaN,67.00,72.00,1.00,NaN,283531.00,100.00,4.00,478.00,2.00,3999.11
max,9999.00,97490.00,909.00,950.00,9901.00,NaN,27004.00,27005.00,157.00,NaN,378041.00,53522.00,80.00,3760000.00,4.00,11369.04


In [ ]:
def predict_valeur_fonciere(row):
    col_pred = ['Nombre_de_lots', 'Surface_reelle_bati', 'Nombre_pieces_principales',
     'month', 'Prix_moyen_m2']

    # ajouter lat et long et niveau de vie à sub_final 

    
    if row['Code_type_local'] == 1:
       pred = model_reg_1.predict(row[col_pred].values.reshape(1, -1))
       return pred
    elif row['Code_type_local'] == 2:
        pred = model_reg_2.predict(row[col_pred].values.reshape(1, -1))
        return pred
    elif row['Code_type_local'] == 3:
        pred = model_reg_3.predict(row[col_pred].values.reshape(1, -1))
        return pred
    elif row['Code_type_local'] == 4:
        pred = model_reg_4.predict(row[col_pred].values.reshape(1, -1))
        return pred

# Appliquer la fonction à chaque ligne du DataFrame
sub_final['TARGET'] = sub_final.apply(predict_valeur_fonciere, axis=1)

sub_final.head()

KeyError: "['latitude', 'longitude', 'niveau_vie_commune'] not in index"

In [ ]:
# les autres variables de la régression dans sub_final 


#######################################



def convertir(liste):
    return liste[0]

# Appliquer la fonction à la colonne 'colonne_liste' à l'aide de apply() et d'une fonction lambda
sub_final['TARGET'] = sub_final['TARGET'].apply(lambda x: convertir(x))


print(sub_final.head())

last_cols = ['ID','TARGET']
sub_final = sub_final[last_cols]

sub_final.head()

  Nature_mutation  No_voie B/T/Q Type_de_voie Code_voie           Voie  \
0           Vente    13.00   NaN          RUE      2280  DE LA LIBERTE   
1           Vente    98.00   NaN          RTE      0055   DE LA DOMBES   
2           Vente   282.00   NaN          RTE      0130    DE POISATON   
3           Vente     7.00   NaN          RUE      0031     DU CORNIER   
4           Vente     7.00   NaN          RUE      3125       DES PINS   

   Code_postal           Commune Code_departement  Code_commune  \
0      1000.00   BOURG EN BRESSE                1            53   
1      1480.00         SAVIGNEUX                1           398   
2      1560.00  MANTENAY MONTLIN                1           230   
3      1150.00     VAUX EN BUGEY                1           431   
4      1000.00   BOURG EN BRESSE                1            53   

   Prefixe_de_section Section  No_plan  No_Volume 1er_lot  \
0                 NaN      AM      102        NaN     7.0   
1                 NaN      ZE 

,ID,TARGET
0,1,53877.38
1,2,222629.53
2,3,165631.65
3,4,160271.24
4,5,196043.82


In [ ]:
sub_final.to_csv('../data/final_submissions.csv')